In [1]:
from xdsl.dialects.experimental import aie
from xdsl.dialects.builtin import Region, IndexType, ModuleOp, i32, IntegerAttr, ArrayAttr, i64, StringAttr
from xdsl.dialects.arith import Constant
from xdsl.dialects.memref import MemRefType
from xdsl.dialects import builtin, arith, memref, func
from xdsl.builder import Builder
from xdsl.traits import SymbolTable

def I32Attr(value):
    return IntegerAttr.from_int_and_width(value, i32)

In [2]:
def tutorial9():
    tile14 = aie.TileOp(I32Attr(1), I32Attr(4))

    buf = aie.BufferOp(tile14, i32, ArrayAttr([I32Attr(256)]), StringAttr("a14"))


    lock14_0 = aie.LockOp(I32Attr(0), I32Attr(0), tile14, StringAttr("lock14_0"))

    kernel = func.FuncOp.external("extern_kernel", [memref.MemRefType.from_element_type_and_shape(i32, [256])], [])
    @Builder.region
    def core14_region(builder: Builder):
        uselock14_0_acquire = aie.UseLockOp(I32Attr(0), I32Attr(aie.LOCK_ACQUIRE), I32Attr(aie.BLOCKING), lock14_0)
        builder.insert(uselock14_0_acquire)
        builder.insert(func.Call(kernel.sym_name.data, [buf], []))
        uselock14_0_release = aie.UseLockOp(I32Attr(1), I32Attr(aie.LOCK_RELEASE), I32Attr(aie.BLOCKING), lock14_0)
        builder.insert(uselock14_0_release)
        builder.insert(aie.EndOp())

    core14 = aie.CoreOp(I32Attr(1), tile14, core14_region, "kernel.o")

    module = ModuleOp([tile14, buf, kernel, lock14_0, core14])
    print(module)

In [3]:
tutorial9()

builtin.module {
  %0 = "AIE.tile"() {"col" = 1 : i32, "row" = 4 : i32} : () -> index
  %1 = "AIE.buffer"(%0) {"sym_name" = "a14"} : (index) -> memref<256xi32>
  func.func private @extern_kernel(memref<256xi32>) -> ()
  %2 = "AIE.lock"(%0) {"lockID" = 0 : i32, "init" = 0 : i32, "sym_name" = "lock14_0"} : (index) -> index
  %3 = AIE.core (%0) {
    AIE.useLock(%2, "Acquire", 0)
    func.call @extern_kernel(%1) : (memref<256xi32>) -> ()
    AIE.useLock(%2, "Release", 1)
    "AIE.end"() : () -> ()
  } { link_with="kernel.o" }
}
